In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs
from reversible2.invert import invert

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs

In [ ]:
orig_train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat')
train_cnt = orig_train_cnt.reorder_channels(['C3', ])

train_inputs = create_inputs(train_cnt, final_hz=64, half_before=True)

In [ ]:
orig_test_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/test/4.mat')
test_cnt = orig_test_cnt.reorder_channels(['C3',])
test_inputs = create_inputs(test_cnt, final_hz=64, half_before=True)

In [ ]:
cuda = True
if cuda:
    train_inputs = [i.cuda() for i in train_inputs]
    test_inputs = [i.cuda() for i in test_inputs]

In [ ]:
from braindecode.datautil.signal_target import SignalAndTarget
train_X = np.concatenate((var_to_np(train_inputs[0]), var_to_np(train_inputs[1])))

train_y = np.concatenate((np.zeros(len(train_inputs[0]), dtype=np.int64),
                          np.ones(len(train_inputs[1]), dtype=np.int64)))

test_X = np.concatenate((var_to_np(test_inputs[0]), var_to_np(test_inputs[1])))

test_y = np.concatenate((np.zeros(len(test_inputs[0]), dtype=np.int64),
                          np.ones(len(test_inputs[1]), dtype=np.int64)))

train_set = SignalAndTarget(train_X, train_y)
valid_set = SignalAndTarget(test_X, test_y)


In [ ]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]

model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=None,
                        pool_time_length=5,
                        pool_time_stride=2,
                        final_conv_length=4)
if cuda:
    model.cuda()
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer,  iterator_seed=1, cropped=True)

In [ ]:
input_time_length = 64
model.fit(train_set.X, train_set.y, epochs=300, batch_size=64, scheduler='cosine',
          input_time_length=input_time_length,
         validation_data=(valid_set.X, valid_set.y),)

In [ ]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.deep4 import Deep4Net
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]

model = Deep4Net(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=None,
                        pool_time_length=2,
                        pool_time_stride=1,
                        final_conv_length=4)
if cuda:
    model.cuda()
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001)
model.compile(loss=F.nll_loss, optimizer=optimizer,  iterator_seed=1, cropped=True)

In [ ]:
input_time_length = 64
model.fit(train_set.X, train_set.y, epochs=300, batch_size=64, scheduler='cosine',
          input_time_length=input_time_length,
         validation_data=(valid_set.X, valid_set.y),)